In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('multas_semaforos.txt')

In [2]:
""" Comprobación de haya filas duplicadas (tenía que comprobarlo aunque dijeras que no había, Juanjo)"""

df.duplicated().sum() 

0

In [3]:
""" Como en algun query nos pide agrupar por calle, y nosotros tenemos en la direccion tanto la calle como
la posicion dentro de esta para indicar una por una cada camara vamos a tener que tratar en detalle esta
parte"""

""" Creamos un dataframe con las direcciones separándolas por espacios dando lugar hasta 5 columnas"""


' Creamos un dataframe con las direcciones separándolas por espacios dando lugar hasta 5 columnas'

In [4]:
direcciones = df['direccion_camara'].str.split(expand=True)
#direcciones.fillna(value=np.nan,inplace=True)
direcciones.head()

,0,1,2,3,4,5
0,1900,N,ASHLAND,AVE,None,None
1,1600,W,CORTLAND,ST,None,None
2,1900,N,ASHLAND,AVE,None,None
3,2400,N,KEDZIE,AVE,None,None
4,848,W,87TH,STREET,None,None


In [5]:
""" La idea sería juntar las columnas 0 y 1 para hacer una columna llamada posicion y juntar las columnas 
2 y 3 para hacer una columna llamada calle, pero tendremos que ver más en detalle esto porque trae problems
"""

' La idea sería juntar las columnas 0 y 1 para hacer una columna llamada posicion y juntar las columnas \n2 y 3 para hacer una columna llamada calle, pero tendremos que ver más en detalle esto porque trae problems\n'

In [6]:
""" Primero comprobamos si algun valor problemático en las columnas 0 y 1, parece que no"""
direcciones[(direcciones[1].isna()) | (direcciones[0].isna())] 

,0,1,2,3,4,5


In [7]:
""" Las columnas 2 y 3 nos dan dos problemas, por un lado vemos que hay terminaciones muy parecidas como ST
y STREET que seguramente darán situaciones en las que nos referimos a la misma calle pero el programa lo 
trataría como calles diferentes.

Por otro lado vemos que tenemos muchos None, o sea que no tiene nada. Esto se soluciona más difícil porque
habrá que comprobar si se refiere a algo que solo son dos palabras o si está incompleto y le faltan las 
últimas columna o si solo es una palabra, para ello tendremos que usar la columna 2"""

' Las columnas 2 y 3 nos dan dos problemas, por un lado vemos que hay terminaciones muy parecidas como ST\ny STREET que seguramente darán situaciones en las que nos referimos a la misma calle pero el programa lo \ntrataría como calles diferentes.\n\nPor otro lado vemos que tenemos muchos None, o sea que no tiene nada. Esto se soluciona más difícil porque habrá\nque comprobar si se refiere a algo que son dos palabras y le falta la última o si solo es una palabra,\npara ello tendremos que usar la columna 2'

In [8]:
""" Solución del primer problema, este casi prefiero ir a mano ya que no es tan largo """

' Solución del primer problema, este casi prefiero ir a mano ya que no es tan largo '

In [10]:
print(pd.unique(direcciones[3]))
direcciones.loc[direcciones[3] == 'ST',3] = 'STREET'
direcciones.loc[direcciones[3] == 'STREE',3] = 'STREET'
direcciones.loc[direcciones[3] == 'AVE',3] = 'AVENUE'
direcciones.loc[direcciones[3] == 'AVEN',3] = 'AVENUE'
direcciones.loc[direcciones[3] == 'AVENU',3] = 'AVENUE'
direcciones.loc[direcciones[3] == 'AVENUE',3] = 'AVENUE'
direcciones.loc[direcciones[3] == 'BLVD',3] = 'BOULEVARD'
direcciones.loc[direcciones[3] == 'BOUL',3] = 'BOULEVARD'
direcciones.loc[direcciones[3] == 'BOULEV',3] = 'BOULEVARD'
direcciones.loc[direcciones[3] == 'RD',3] = 'ROAD'
direcciones.loc[direcciones[3] == 'PK',3] = 'PARKWAY'
direcciones.loc[direcciones[3] == 'PARK',3] = 'PARKWAY'
direcciones.loc[direcciones[3] == 'PARKWA',3] = 'PARKWAY'
direcciones.loc[direcciones[3] == 'DRIV',3] = 'DRIVE'
print(pd.unique(direcciones[3]))

['AVE' 'ST' 'STREET' None 'AVENUE' 'RD' 'PK' 'GROVE' 'BLVD' 'CHICAGO'
 'ISLAND' 'AVENU' 'ROAD' 'DRIVE' 'AVEN' 'BUREN' 'STREE' 'MARTIN' 'DRIV'
 'PARKWAY' 'PARK' 'BOULEVARD' 'BOUL' 'BOULEV' 'PARKWA']
['AVENUE' 'STREET' None 'ROAD' 'PARKWAY' 'GROVE' 'BOULEVARD' 'CHICAGO'
 'ISLAND' 'DRIVE' 'BUREN' 'MARTIN']


In [12]:
print(pd.unique(direcciones[4]))
direcciones.loc[direcciones[4] == 'STREE',4] = 'STREET'
direcciones.loc[direcciones[4] == 'AVE',4] = 'AVENUE'
direcciones.loc[direcciones[4] == 'AV',4] = 'AVENUE'
direcciones.loc[direcciones[4] == 'A',4] = 'AVENUE'
direcciones.loc[direcciones[4] == 'ROA',4] = 'ROAD'
direcciones.loc[direcciones[4] == 'RD',4] = 'ROAD'
print(pd.unique(direcciones[4]))

[None 'AVE' 'A' 'STREE' 'L' 'ROAD' 'ROA' 'AV' 'RD']
[None 'AVENUE' 'STREET' 'L' 'ROAD']


In [13]:
""" Solucionado en la columna 4, nos queda ver la 5, que podemos ver que está decente """

' Solucionado en la columna 4, nos queda ver la 5, que podemos ver que está decente '

In [14]:
pd.unique(direcciones[5])

array([None, 'KING'], dtype=object)

In [15]:
""" Ahora toca solucionar problemas de falta de palabras. Primero miro aquellas filas en las que la calle
está formada por una sola palabra (e.g. MADISON EN VEZ DE MADISON STREET) para comparar con filas en las 
que sí está completo y poder rellenar correctamente"""

' Ahora toca solucionar problemas de falta de palabras. Primero miro aquellas filas en las que la calle\nestá formada por una sola palabra (e.g. MADISON EN VEZ DE MADISON STREET) para comparar con filas en las \nque sí está completo y poder rellenar correctamente'

In [16]:
incompleto = pd.unique(direcciones[direcciones[3].isna()][2])
incompleto

array(['DIVISION', 'HALSTED', 'MADISON', 'CALIFORNIA', '35TH', 'NORTH',
       'PULASKI', '71ST', 'BELMONT', 'VINCENNES', 'PETERSON', 'ASHLAND',
       'DIVERSEY', 'LASALLE', 'KOSTNER', 'NAGLE', 'CERMAK', '31ST',
       'NARRAGANSETT', 'ROOSEVELT', 'WESTERN', 'KINZIE', 'FOSTER',
       'ADDISON'], dtype=object)

In [18]:
""" Bueno pues se ve en la lista que muchas se rellenan fácil con avenue, road o street y otras con nada.
Pero hay una que se puede rellenar con 2, WESTERN AVENUE o WESTERN BOULEVARD (bieeeen). He buscado en maps
(menos mal que leí que el artículo era de Chicago) y son dos calles paralelas igual de largas más o menos,
así que creo que lo más justo sería rellenar la mitad de los WESTERN con WESTERN AVENUE y la otra mitad con
WESTERN BOULEVARD ya solo por rizar el rizo. Con DIVERSEY ocurre algo parecido así que haré lo mismo.

Con PULASKI también pero de las 113000 no vacías 109000 van con ROAD así que las añado a ROAD"""

' Bueno pues se ve en la lista que muchas se rellenan fácil con avenue, road o street y otras con nada.\nPero hay una que se puede rellenar con 2, WESTERN AVENUE o WESTERN BOULEVARD (bieeeen). He buscado en maps\n(menos mal que leí que el artículo era de Chicago) y son dos calles paralelas igual de largas más o menos,\nasí que creo que lo más justo sería rellenar la mitad de los WESTERN con WESTERN AVENUE y la otra mitad con\nWESTERN BOULEVARD ya solo por rizar el rizo. Con DIVERSEY ocurre algo parecido así que haré lo mismo.\n\nCon PULASKI también pero de las 113000 no vacías 109000 van con ROAD así  que las añado a ROAD'

In [19]:
""" Sé que es muy cutre pero lo voy a hacer a mano una a una porque da bastantes fallos raros y estoy 
haciendo cambios directos al dataframe. Además, hay algunas localizaciones formadas por una palabra que
voy a rellenar a mano buscando en maps los que son como Nagle, o 35TH que esa pide claramente STREET y no
lo tiene"""

' Sé que es muy cutre pero lo voy a hacer a mano una a una porque da bastantes fallos raros y estoy \nhaciendo cambios directos al dataframe. Además, hay algunas localizaciones formadas por una palabra que\nvoy a rellenar a mano buscando en maps los que son como Nagle, o 35TH que esa pide claramente STREET y no\nlo tiene'

In [20]:
direcciones.loc[(direcciones[2]=='DIVISION') & (direcciones[3].isna()),3] = 'STREET'
direcciones.loc[(direcciones[2]=='HALSTED') & (direcciones[3].isna()),3] = 'STREET'
direcciones.loc[(direcciones[2]=='MADISON') & (direcciones[3].isna()),3] = 'STREET'
direcciones.loc[(direcciones[2]=='CALIFORNIA') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='35TH') & (direcciones[3].isna()),3] = 'STREET'
direcciones.loc[(direcciones[2]=='NORTH') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='PULASKI') & (direcciones[3].isna()),3] = 'ROAD'
direcciones.loc[(direcciones[2]=='71ST') & (direcciones[3].isna()),3] = 'STREET'
direcciones.loc[(direcciones[2]=='BELMONT') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='VINCENNES') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='PETERSON') & (direcciones[3].isna()),3] ='AVENUE'
direcciones.loc[(direcciones[2]=='ASHLAND') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='LASALLE') & (direcciones[3].isna()),3] = 'STREET'
direcciones.loc[(direcciones[2]=='KOSTNER') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='NAGLE') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='CERMAK') & (direcciones[3].isna()),3] = 'ROAD'
direcciones.loc[(direcciones[2]=='31ST') & (direcciones[3].isna()),3] = 'STREET'
direcciones.loc[(direcciones[2]=='NARRAGANSETT') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='ROOSEVELT') & (direcciones[3].isna()),3] = 'ROAD'
direcciones.loc[(direcciones[2]=='KINZIE') & (direcciones[3].isna()),3] = 'STREET'
direcciones.loc[(direcciones[2]=='FOSTER') & (direcciones[3].isna()),3] = 'AVENUE'
direcciones.loc[(direcciones[2]=='ADDISON') & (direcciones[3].isna()),3] = 'STREET'

In [21]:
""" Veamos que hacer con las dos especiales qe hay que repartir a la mitad. Primero encuentro sus posiciones
en la dataframe"""

' Veamos que hacer con las dos especiales qe hay que repartir a la mitad. Primero encuentro sus posiciones\nen la dataframe'

In [22]:
indice_div = direcciones.loc[(direcciones[2]=='DIVERSEY') & (direcciones[3].isna()),3].index
indice_wes = direcciones.loc[(direcciones[2]=='WESTERN') & (direcciones[3].isna()),3].index

In [23]:
direcciones.iloc[indice_div[:int(len(indice_div)/2)],3] = 'PARKWAY'
direcciones.iloc[indice_div[int(len(indice_div)/2):],3] = 'AVENUE'
direcciones.iloc[indice_wes[int(len(indice_wes)/2):],3] = 'AVENUE'
direcciones.iloc[indice_wes[:int(len(indice_wes)/2)],3] = 'BOULEVARD'

In [24]:
""" Todo este relleno se ha hecho sabiendo lo que toca por líneas de código que he borrado, no considero 
necesario mostrar esa parte pero aseguro que no se ha rellenado al azar. Podemos ver que ya no hay valores
vacíos en la columna 2, hemos completado bien los nombres (al menos eso creo)"""

' Todo este relleno se ha hecho sabiendo lo que toca por líneas de código que he borrado, no considero \nnecesario mostrar esa parte pero aseguro que no se ha rellenado al azar. Podemos ver que ya no hay valores\nvacíos en la columna 2, hemos completado bien los nombres (al menos eso creo)'

In [25]:
pd.unique(direcciones[direcciones[3].isna()][2])

array([], dtype=object)

In [26]:
""" Ahora toca crear las columnas bien a partir de los datos correctos. Creamos un dataframe aparte para 
tenerlo todo bien localizado. Primero creamos una columna que es la posición en la calle a partir de las
columnas 0 y 1 y luego una segunda columna que será la calle a partir de las columnas 2, 3, 4 y 5"""

' Ahora toca crear las columnas bien a partir de los datos correctos. Creamos un dataframe aparte para \ntenerlo todo bien localizado. Primero creamos una columna que es la posición en la calle a partir de las\ncolumnas 0 y 1 y luego una segunda columna que será la calle a partir de las columnas 2, 3, 4 y 5'

In [27]:
direcciones_bien = pd.DataFrame()

In [28]:
direcciones_bien["posicion"] = direcciones[0]+ " " + direcciones[1]

In [29]:
""" Ahora toca crear una segunda columna con el nombre de la calle"""

' Ahora toca crear una segunda columna con el nombre de la calle'

In [30]:
direcciones_bien["calle"] = direcciones[2] + (' ' + direcciones[3]).fillna('') + \
                            (' ' + direcciones[4]).fillna('') + (' ' + direcciones[5]).fillna('') 

In [31]:
direcciones_bien

,posicion,calle
0,1900 N,ASHLAND AVENUE
1,1600 W,CORTLAND STREET
2,1900 N,ASHLAND AVENUE
3,2400 N,KEDZIE AVENUE
4,848 W,87TH STREET
...,...,...
4174765,2000 W,DIVISION STREET
4174766,9900 S,HALSTED STREET
4174767,5500 S,PULASKI ROAD
4174768,2400 N,ASHLAND AVENUE


In [32]:
df.head()

,multa_id,fecha_hora,matricula,tipo_placa,estado_placa,direccion_camara
0,7000634986,2007-01-01 00:02:00,Y301669,PAS,IL,1900 N ASHLAND AVE
1,7000634946,2007-01-01 00:03:00,4703TX,TXI,IL,1600 W CORTLAND ST
2,7000634987,2007-01-01 00:14:00,9125630,PAS,IL,1900 N ASHLAND AVE
3,7000634926,2007-01-01 00:30:00,220H070,TMP,IL,2400 N KEDZIE AVE
4,7000635682,2007-01-01 00:43:00,4875778,PAS,IL,848 W 87TH STREET


In [33]:
""" Pues ya parece estar todo medio solucionado con las direcciones, pasemos a mirar otras cosas"""

' Pues ya parece estar todo medio solucionado con las direcciones, pasemos a mirar otras cosas'

In [34]:
""" La columna de fecha_hora la cambio al formato datetime para trabajar con ella mejor"""

df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])

In [35]:
""" Creo dos nuevas columnas, fecha y hora por separado ya que toca hacer alguna clasificación
    por días y es mejor tenerlo separado"""

df['fecha'] = [d.date() for d in df['fecha_hora']]
df['hora'] = [d.time() for d in df['fecha_hora']]

In [36]:
df["posicion"] = direcciones_bien["posicion"]
df["calle"] = direcciones_bien["calle"]
df.head()

,multa_id,fecha_hora,matricula,tipo_placa,estado_placa,direccion_camara,fecha,hora,posicion,calle
0,7000634986,2007-01-01 00:02:00,Y301669,PAS,IL,1900 N ASHLAND AVE,2007-01-01,00:02:00,1900 N,ASHLAND AVENUE
1,7000634946,2007-01-01 00:03:00,4703TX,TXI,IL,1600 W CORTLAND ST,2007-01-01,00:03:00,1600 W,CORTLAND STREET
2,7000634987,2007-01-01 00:14:00,9125630,PAS,IL,1900 N ASHLAND AVE,2007-01-01,00:14:00,1900 N,ASHLAND AVENUE
3,7000634926,2007-01-01 00:30:00,220H070,TMP,IL,2400 N KEDZIE AVE,2007-01-01,00:30:00,2400 N,KEDZIE AVENUE
4,7000635682,2007-01-01 00:43:00,4875778,PAS,IL,848 W 87TH STREET,2007-01-01,00:43:00,848 W,87TH STREET


In [37]:
""" Selecciono solo las columnas útiles para hacer más fácil la query. He quitado también
    la ID de las multas porque ya nos hemos cerciorado que no hay duplicados y porque no se nos 
    pide identificar una multa determinada en ningún momento"""

final_df = df[['matricula','fecha','hora','posicion','calle',]]
final_df.head()

,matricula,fecha,hora,posicion,calle
0,Y301669,2007-01-01,00:02:00,1900 N,ASHLAND AVENUE
1,4703TX,2007-01-01,00:03:00,1600 W,CORTLAND STREET
2,9125630,2007-01-01,00:14:00,1900 N,ASHLAND AVENUE
3,220H070,2007-01-01,00:30:00,2400 N,KEDZIE AVENUE
4,4875778,2007-01-01,00:43:00,848 W,87TH STREET


In [ ]:
""" Ya tenemos unos datos decentes y con las columnas que vamos a utilizar, sin información de más y la que
hay nos hemos asegurados de que sea la correcta"""

In [70]:
""" Por último nos queda pasar el dataframe a otro csv que será con el que trabajemos en Pig y Hive"""
final_df.to_csv('multas_semaforos_mod.txt',index=False)